# <center>Дисциплина "Эконометрика"</center>
# <center>Дополнение к эссе:<br>процесс программного сбора данных с помощью технологии Web Scraping</center>
## <center>Выполнила: Кузнецова Анастасия, группа 18Э1</center>

Начнём с импортирования необходимых модулей и библиотек:
* **`re`** - обработка регулярных выражений для извлечения необходимой информации из html-кода страницы  сайта;
* **`pandas`** - программная библиотека для обработки и анализа данных, необходима для создания и управления таблицами с данными в настоящей работе;
* **`BeautifulSoup`** - парсер для синтаксического разбора файлов HTML;
* **`selenium webdriver`** - программная библиотека для управления браузерами.

In [1]:
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

Создаём пустой датафрейм, в котором будут храниться выгружаемые данные.

In [86]:
df = pd.DataFrame()

Также создадим несколько пустых списков для того, чтобы зафиксировать основную информацию, которая предоставляется в ходе поиска автомобиля по заданным критериям на сайте **`auto.ru`**.

In [87]:
years = [] # Год выпуска автомобиля
dists = [] # Пробег автомобиля, км
links = [] # Ссылка на карточку объявления
names = [] # Название автомобиля
drive_char_1 = [] # Рабочий объём двигателя, л
drive_char_2 = [] # Мощность двигателя, количество лошадиных сил
drive_char_3 = [] # Моторное топливо
drive_char_4 = [] # Вид трансмиссии
drive_char_5 = [] # Тип кузова 
drive_char_6 = [] # Тип привода 
drive_char_7 = [] # Цвет автомобиля
prices = [] # Цена автомобиля, руб.
places = [] # Геолокация продавца

Функция поиска и выгрузки основной информации карточек:

In [126]:
def data_capture(lnk, l_i): # принимает на вход ссылку на стратовую страницу, а также количество страниц после поиска
    for p in range(1, l_i + 1): # цикл, чтобы охватить все страницы

        URL = lnk + str(p) # ссылка + номер странички
        # запускаем отдельное автоматизированное окно в браузере Google Chrome:
        browser = webdriver.Chrome(executable_path='C:/Users/paxom/Documents/IAD/PycharmProjects/IO/chromedriver.exe')
        browser.get(URL)
        # парсим страничку при помощи BeautifulSoup:
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        # вручную ищем атрибут, который хранит по отдельности каждую из мини-карточек (рис. 1), 
        # выгружаем все эти карточки и информацию, которая содержится в них, в список:
        results = soup.find_all('div', {'class': 'ListingItem__description'})
        
        for i in range(len(results)): # выгружаем необходимые данные из каждой карточки
            try:
                price = results[i].select("div[class='ListingItemPrice__content']") # ищем атрибут,  где хранится цена (рис. 2)
                prices.append(int(''.join(re.split('\xa0|₽', price[0].text.strip())))) # очищаем данные от мусора, записываем результат в список
                year = results[i].select("div[class='ListingItem__year']") # аналогично выбираем остальную информацию
                years.append(int(year[0].text.strip()))
                dist = results[i].select("div[class='ListingItem__kmAge']")
                dists.append(dist[0].text.strip().replace('\xa0', ' '))
                link = results[i].select("a[class='Link ListingItemTitle__link']")
                links.append(re.split('"', str(link[0]))[3])
                names.append(link[0].text.strip())
                descr = results[i].select("div[class='ListingItemTechSummaryDesktop ListingItem__techSummary']")
#                 char = re.split('\u2009|\xa0| л', descr[0].text.strip())
                items_tr_list = [i for i in re.split('<div class="ListingItemTechSummaryDesktop__cell">|</div>|<div class="ListingItemTechSummaryDesktop__column">|<div class="ListingItemTechSummaryDesktop ListingItem__techSummary">', 
                                                     str(descr[0])) if len(i) > 0]
                items_tr_list_1 = []
                for itr in items_tr_list:
                    tr = re.split('\u2009|/', itr)
                    for r in tr:
                        if len(r) > 0:
                            items_tr_list_1.append(r.replace('\xa0', ' '))
                
                drive_char_1.append(items_tr_list_1[0])
                drive_char_2.append(items_tr_list_1[1])
                drive_char_3.append(items_tr_list_1[2])
                drive_char_4.append(items_tr_list_1[3])
                drive_char_5.append(items_tr_list_1[4])
                drive_char_6.append(items_tr_list_1[5])
                drive_char_7.append(items_tr_list_1[6])
                place = results[i].select("span[class='MetroListPlace__regionName MetroListPlace_nbsp']")
                places.append(' '.join(re.split('\xa0', place[0].text.strip())))
            except IndexError:
                continue
        
        # закрываем отдельное автоматизированное окно в браузере Google Chrome:
        browser.quit()

<img src="Data_browse.Kuznetsova/Code_img_1.png" width="1000" height="600">
<br>
<center><i>Рис. 1. Пример мини-карточки автомобиля в поиске. Атрибут html-кода, который хранит основную информацию карточки автомобиля<i><center>

<img src="Data_browse.Kuznetsova/Code_img_2.png" width="1000" height="600">
<br>
<center><i>Рис. 2. Атрибут html-кода, который хранит информацию о цене автомобиля<i><center>

Находим данные по нескольким моделям автомобилей со схожими характеристиками и ценовым диапазоном, выгружаем данные.

In [127]:
data_capture('https://auto.ru/kovrov/cars/renault/sandero/used/?page=', 3)
# data_capture('https://auto.ru/kovrov/cars/citroen/c3_picasso/used/?page=', 1)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=96.0.4664.45)


(Я сначала не заметила, что два раза выгрузила данные по Citroen C3 Picasso, в самом конце удалила вручную).

In [6]:
# data_capture('https://auto.ru/kovrov/cars/citroen/c3_picasso/used/?page=', 1)
data_capture('https://auto.ru/kovrov/cars/ford/fusion/used/?page=', 2)
data_capture('https://auto.ru/kovrov/cars/honda/jazz/used/?page=', 1)
data_capture('https://auto.ru/kovrov/cars/nissan/note/used/?page=', 1)
data_capture('https://auto.ru/kovrov/cars/suzuki/sx4/used/?page=', 1)

Формируем из получившихся данных таблицу.

In [90]:
df['links'] = links
df['names'] = names
df['years'] = years
df['dists'] = dists
df['prices'] = prices
df['drive_char_1'] = drive_char_1
df['drive_char_2'] = drive_char_2
df['drive_char_3'] = drive_char_3
df['drive_char_4'] = drive_char_4
df['drive_char_5'] = drive_char_5
df['drive_char_6'] = drive_char_6
df['drive_char_7'] = drive_char_7
df['places'] = places

Сохраняем таблицу в формате .csv (чтобы в случае чего, не потерять промежуточную информацию).

In [8]:
df.to_csv('cars.csv', sep=';', encoding="utf-8-sig", index=False)

Посмотрим, правильно ли рапределились типы данных (вроде всё в норме).

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   links         227 non-null    object 
 1   names         227 non-null    object 
 2   years         227 non-null    int64  
 3   dists         227 non-null    int64  
 4   prices        227 non-null    int64  
 5   drive_char_1  227 non-null    float64
 6   drive_char_2  227 non-null    object 
 7   drive_char_3  227 non-null    object 
 8   drive_char_4  227 non-null    object 
 9   drive_char_5  227 non-null    object 
 10  drive_char_6  227 non-null    object 
 11  drive_char_7  227 non-null    object 
 12  places        227 non-null    object 
dtypes: float64(1), int64(3), object(9)
memory usage: 23.2+ KB


In [10]:
df = pd.read_csv('cars.csv', sep=';', encoding="utf-8-sig")

Получившаяся таблица выглядит следующим образом:

In [91]:
df

,links,names,years,dists,prices,drive_char_1,drive_char_2,drive_char_3,drive_char_4,drive_char_5,drive_char_6,drive_char_7,places


Попробуем выгрузить дополнительные данные по каждому автомобилю.
<br>
<br>Импортируем модуль транслитерации, чтобы впоследствии не заморачиваться с названиями признаков и переменных (они будут автоматически переписаны при помощи букв английского алфавита).

In [12]:
import transliterate

Полученные данные запишем в словарь.

In [15]:
char_dict = {}

In [16]:
for i in range(df.shape[0]):
    URL = df['links'][i] # открываем каждую ссылку объявления
    # запускаем отдельное автоматизированное окно в браузере Google Chrome:
    browser = webdriver.Chrome(executable_path='C:/Users/paxom/Documents/IAD/PycharmProjects/IO/chromedriver.exe')
    browser.get(URL)
    # парсим страничку при помощи BeautifulSoup:
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    # вручную ищем атрибуты, которые хранят информацию о комплектации автомобиля (рис. 3)
    results = soup.find_all('div', {'class': 'ComplectationGroups__group'})
    # 
    char_list = []
    for r in range(len(results)): 
        # название категории комплектации (извлекаем, затем переписываем при помощи транслитерации):
        name = transliterate.translit(results[r].select("span[class='ComplectationGroups__itemName']")[0].text.strip(), reversed=True)
        # составляющие категории комплектации:
        data = [results[r].select("li[class='ComplectationGroups__itemContentEl']")[j].text.strip() 
                for j in range(len(results[r].select("li[class='ComplectationGroups__itemContentEl']")))]
        # проверяем наличие категории в ключах словаря, если её нет то добавляем новую + данные
        # составляющих категории комплектации (к данным добавляем еще порядковый номер объявления, 
        # чтобы не запутаться, какие характеристики к каким автомобилям относятся):
        if name not in list(char_dict.keys()):
            char_dict[str(name)] = [str(i) + ' ' + ', '.join(data)]
        else:
            trial = char_dict.get(str(name))
            char_dict.update({str(name): trial + [str(i) + ' ' + ', '.join(data)]})
            
    # закрываем отдельное автоматизированное окно в браузере Google Chrome:
    browser.quit()

<img src="Data_browse.Kuznetsova/Code_img_3.png" width="1000" height="600">
<br>
<center><i>Рис. 3. Пример атрибута html-кода, который хранит информацию о комплектации автомобиля<i><center>

На всякий случай сохраним полученный словарь с характеристиками комплектующих при помощи библиотеки **`numpy`**.

In [41]:
import numpy as np

In [42]:
# Save
# np.save('my_file.npy', char_dict) 

In [57]:
# Load
char_dict = np.load('my_file.npy', allow_pickle='TRUE').item()

Необходимо обработать полученные данные (так как в некоторых моделях отсутствуют различные категории дополнительных характеристик) и сформировать новую таблицу, состоящую лишь из дополнительных данных по автомобилям. Создаем новый пустой словарь.

In [59]:
char_dict1 = {}

Необходимо проидентифицировать объявления, в которых нет данных по некоторым категориям.

In [60]:
for key in list(char_dict.keys()): # проходимся по каждому ключу - категории дополнительных характеристик
    item_list = char_dict.get(key) # извлекаем все значения определенного ключа
    num_list = [int(i.split(' ')[0]) for i in char_dict.get(key)] # ищем список всех номеров объявлений, у которых есть записи по категории
    missed = list(set(range(df.shape[0])) - set(num_list)) # ищем номера тех объявлений, где нет записей
    for i in missed:
        item_list.insert(i, str(i)) # вставляем только порядковые номера объявлений, в которых не указаны доп. хар-ки в категории
    char_dict1[key] = item_list # обновим значения в категории

Сформируем датафрейм со всеми категорями комплектации и данными по ним.

In [65]:
df_char = pd.DataFrame(char_dict1)

Из каждой записи удаляем порядковый номер объявления.

In [80]:
for c in df_char.columns:
    df_char[c] = df_char[c].apply(lambda x: ' '.join(x.split(' ')[1:]))

Получившиеся незаполненные ячейки меняем на пустые - NaN.

In [88]:
df_char = df_char.replace({'': np.nan})

Готовая табблица дополнительных характеристик:

In [89]:
df_char

,Komfort,Elementy ekster'era,Salon,Mul'timedia,Obzor,Zaschita ot ugona,Bezopasnost',Prochee
0,"Круиз-контроль, Электростеклоподъёмники задние...",Легкосплавные диски,Подогрев передних сидений,"USB, AUX, Аудиосистема",Электрообогрев лобового стекла,NaN,NaN,NaN
1,"Круиз-контроль, Электростеклоподъёмники передн...",Легкосплавные диски,Подогрев передних сидений,"USB, AUX, Аудиосистема",NaN,Сигнализация,NaN,NaN
2,NaN,Стальные диски,"Складывающееся заднее сиденье, Ткань (Материал...",Аудиоподготовка,NaN,Иммобилайзер,"Подушка безопасности водителя, Крепление детск...",NaN
3,"Электростеклоподъёмники задние, Кондиционер",NaN,Подогрев передних сидений,"USB, AUX, Аудиосистема",NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223,"Электростеклоподъёмники передние, Мультифункци...","Рейлинги на крыше, Стальные диски, Декоративны...","Регулировка передних сидений по высоте, Склады...","AUX, USB, Мультимедиа система с ЖК-экраном, На...","Дневные ходовые огни, Противотуманные фары, Эл...","Центральный замок, Сигнализация с обратной свя...","Подушка безопасности водителя, Крепление детск...",NaN
224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
225,"Электростеклоподъёмники передние, Мультифункци...","Рейлинги на крыше, Диски 16, Декоративные молд...","Складывающееся заднее сиденье, Ткань (Материал...",Аудиосистема,"Противотуманные фары, Электрообогрев боковых з...","Центральный замок, Иммобилайзер","Система стабилизации (ESP), Подушка безопаснос...",NaN


Информация по полученной таблице:

In [90]:
df_char.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Komfort              150 non-null    object
 1   Elementy ekster'era  94 non-null     object
 2   Salon                131 non-null    object
 3   Mul'timedia          123 non-null    object
 4   Obzor                108 non-null    object
 5   Zaschita ot ugona    125 non-null    object
 6   Bezopasnost'         117 non-null    object
 7   Prochee              60 non-null     object
dtypes: object(8)
memory usage: 14.3+ KB


На всякий случай сохраним датафрейм.

In [91]:
df_char.to_csv('cars_chars.csv', sep=';', encoding="utf-8-sig", index=False)

In [92]:
df_char = pd.read_csv('cars_chars.csv', sep=';', encoding="utf-8-sig")

In [93]:
df_char['index_col'] = df_char.index

In [94]:
df_char

,Komfort,Elementy ekster'era,Salon,Mul'timedia,Obzor,Zaschita ot ugona,Bezopasnost',Prochee,index_col
0,"Круиз-контроль, Электростеклоподъёмники задние...",Легкосплавные диски,Подогрев передних сидений,"USB, AUX, Аудиосистема",Электрообогрев лобового стекла,NaN,NaN,NaN,0
1,"Круиз-контроль, Электростеклоподъёмники передн...",Легкосплавные диски,Подогрев передних сидений,"USB, AUX, Аудиосистема",NaN,Сигнализация,NaN,NaN,1
2,NaN,Стальные диски,"Складывающееся заднее сиденье, Ткань (Материал...",Аудиоподготовка,NaN,Иммобилайзер,"Подушка безопасности водителя, Крепление детск...",NaN,2
3,"Электростеклоподъёмники задние, Кондиционер",NaN,Подогрев передних сидений,"USB, AUX, Аудиосистема",NaN,NaN,NaN,NaN,3
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...
222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,222
223,"Электростеклоподъёмники передние, Мультифункци...","Рейлинги на крыше, Стальные диски, Декоративны...","Регулировка передних сидений по высоте, Склады...","AUX, USB, Мультимедиа система с ЖК-экраном, На...","Дневные ходовые огни, Противотуманные фары, Эл...","Центральный замок, Сигнализация с обратной свя...","Подушка безопасности водителя, Крепление детск...",NaN,223
224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224
225,"Электростеклоподъёмники передние, Мультифункци...","Рейлинги на крыше, Диски 16, Декоративные молд...","Складывающееся заднее сиденье, Ткань (Материал...",Аудиосистема,"Противотуманные фары, Электрообогрев боковых з...","Центральный замок, Иммобилайзер","Система стабилизации (ESP), Подушка безопаснос...",NaN,225


Думаю, логично создать из каждой характеристики отдельную категорию с бинарными переменными.

In [108]:
for c in df_char.columns[:-1]:
    # из каждой категории вытягиваем характеристики, чтобы сформировать dummy-переменные:
    cleaned = df_char.set_index('index_col')[c].str.split(', ', expand=True).stack()
    # добавляем dummy-переменные к исходной таблице, предварительно записав их с префиксом, 
    # означающим категорию (например, P - прочее):
    df = pd.concat([df, pd.get_dummies(cleaned, prefix=c[0]).groupby(level=0).sum()], axis=1) 

Получившаяся таблица:

In [109]:
df

,links,names,years,dists,prices,drive_char_1,drive_char_2,drive_char_3,drive_char_4,drive_char_5,...,B_Система помощи при старте в гору (HSA),B_Система помощи при торможении (BAS,B_Система стабилизации (ESP),B_ЭРА-ГЛОНАСС,P_Активная подвеска,P_Газобаллонное оборудование,P_Докатка,P_Защита картера,P_Полноразмерное запасное колесо,P_Фаркоп
0,https://auto.ru/cars/used/sale/renault/sandero...,Renault Sandero II Stepway,2018,69800,769000,1.6,113,Бензин,механика,хэтчбек 5 дв.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://auto.ru/cars/used/sale/renault/sandero...,Renault Sandero II Stepway,2017,21166,769000,1.6,82,Бензин,механика,хэтчбек 5 дв.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://auto.ru/cars/used/sale/renault/sandero...,Renault Sandero I,2014,104000,299999,1.4,75,Бензин,механика,хэтчбек 5 дв.,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,https://auto.ru/cars/used/sale/renault/sandero...,Renault Sandero I,2011,128169,319000,1.6,84,Бензин,механика,хэтчбек 5 дв.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://auto.ru/cars/used/sale/renault/sandero...,Renault Sandero I,2013,69300,350000,1.4,75,Бензин,механика,хэтчбек 5 дв.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,https://auto.ru/cars/used/sale/suzuki/sx4/1101...,Suzuki SX4 I (Classic) Рестайлинг,2011,148000,520000,1.6,112,Бензин,механика,хэтчбек 5 дв.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223,https://auto.ru/cars/used/sale/suzuki/sx4/1102...,Suzuki SX4 I (Classic) Рестайлинг,2012,112000,712000,1.6,112,Бензин,автомат,хэтчбек 5 дв.,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
224,https://auto.ru/cars/used/sale/suzuki/sx4/1102...,Suzuki SX4 I (Classic) Рестайлинг,2012,108000,630399,1.6,112,Бензин,автомат,хэтчбек 5 дв.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
225,https://auto.ru/cars/used/sale/suzuki/sx4/1094...,Suzuki SX4 I (Classic) Рестайлинг,2013,130500,700000,1.6,112,Бензин,автомат,хэтчбек 5 дв.,...,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


Заполним все пропущенные значения нулями.

In [113]:
df = df.fillna(0)

In [115]:
df.to_csv('cars_all.csv', sep=';', encoding="utf-8-sig", index=False)

Удалим колонки, в которых значений "1" или "0" менее 10 штук.

In [138]:
del_cols = []

In [139]:
for col in df.columns[13:]:
    if df[col].value_counts()[0] <= 10 or df[col].value_counts()[1] <= 10:
        del_cols.append(col)

In [141]:
df = df.drop(del_cols, axis=1)

In [142]:
df

,links,names,years,dists,prices,drive_char_1,drive_char_2,drive_char_3,drive_char_4,drive_char_5,...,B_Антиблокировочная система (ABS),B_Блокировка замков задних дверей,B_Крепление детского кресла (задний ряд) ISOFIX,B_Подушка безопасности водителя,B_Подушка безопасности пассажира,B_Подушки безопасности боковые,B_Система помощи при торможении (BAS,B_Система стабилизации (ESP),P_Защита картера,P_Полноразмерное запасное колесо
0,https://auto.ru/cars/used/sale/renault/sandero...,Renault Sandero II Stepway,2018,69800,769000,1.6,113,Бензин,механика,хэтчбек 5 дв.,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,https://auto.ru/cars/used/sale/renault/sandero...,Renault Sandero II Stepway,2017,21166,769000,1.6,82,Бензин,механика,хэтчбек 5 дв.,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,https://auto.ru/cars/used/sale/renault/sandero...,Renault Sandero I,2014,104000,299999,1.4,75,Бензин,механика,хэтчбек 5 дв.,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,https://auto.ru/cars/used/sale/renault/sandero...,Renault Sandero I,2011,128169,319000,1.6,84,Бензин,механика,хэтчбек 5 дв.,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,https://auto.ru/cars/used/sale/renault/sandero...,Renault Sandero I,2013,69300,350000,1.4,75,Бензин,механика,хэтчбек 5 дв.,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,https://auto.ru/cars/used/sale/suzuki/sx4/1101...,Suzuki SX4 I (Classic) Рестайлинг,2011,148000,520000,1.6,112,Бензин,механика,хэтчбек 5 дв.,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
223,https://auto.ru/cars/used/sale/suzuki/sx4/1102...,Suzuki SX4 I (Classic) Рестайлинг,2012,112000,712000,1.6,112,Бензин,автомат,хэтчбек 5 дв.,...,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
224,https://auto.ru/cars/used/sale/suzuki/sx4/1102...,Suzuki SX4 I (Classic) Рестайлинг,2012,108000,630399,1.6,112,Бензин,автомат,хэтчбек 5 дв.,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
225,https://auto.ru/cars/used/sale/suzuki/sx4/1094...,Suzuki SX4 I (Classic) Рестайлинг,2013,130500,700000,1.6,112,Бензин,автомат,хэтчбек 5 дв.,...,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0


Преобразуем другие признаки в бинарный тип.

In [146]:
df = df.replace({'Бензин': 0, 'Дизель': 1})

In [150]:
df = df.replace({'передний': 0, 'полный': 1})

In [152]:
df

,links,names,years,dists,prices,drive_char_1,drive_char_2,drive_char_3,drive_char_4,drive_char_5,...,B_Антиблокировочная система (ABS),B_Блокировка замков задних дверей,B_Крепление детского кресла (задний ряд) ISOFIX,B_Подушка безопасности водителя,B_Подушка безопасности пассажира,B_Подушки безопасности боковые,B_Система помощи при торможении (BAS,B_Система стабилизации (ESP),P_Защита картера,P_Полноразмерное запасное колесо
0,https://auto.ru/cars/used/sale/renault/sandero...,Renault Sandero II Stepway,2018,69800,769000,1.6,113,0,механика,хэтчбек 5 дв.,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,https://auto.ru/cars/used/sale/renault/sandero...,Renault Sandero II Stepway,2017,21166,769000,1.6,82,0,механика,хэтчбек 5 дв.,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,https://auto.ru/cars/used/sale/renault/sandero...,Renault Sandero I,2014,104000,299999,1.4,75,0,механика,хэтчбек 5 дв.,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,https://auto.ru/cars/used/sale/renault/sandero...,Renault Sandero I,2011,128169,319000,1.6,84,0,механика,хэтчбек 5 дв.,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,https://auto.ru/cars/used/sale/renault/sandero...,Renault Sandero I,2013,69300,350000,1.4,75,0,механика,хэтчбек 5 дв.,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,https://auto.ru/cars/used/sale/suzuki/sx4/1101...,Suzuki SX4 I (Classic) Рестайлинг,2011,148000,520000,1.6,112,0,механика,хэтчбек 5 дв.,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
223,https://auto.ru/cars/used/sale/suzuki/sx4/1102...,Suzuki SX4 I (Classic) Рестайлинг,2012,112000,712000,1.6,112,0,автомат,хэтчбек 5 дв.,...,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
224,https://auto.ru/cars/used/sale/suzuki/sx4/1102...,Suzuki SX4 I (Classic) Рестайлинг,2012,108000,630399,1.6,112,0,автомат,хэтчбек 5 дв.,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
225,https://auto.ru/cars/used/sale/suzuki/sx4/1094...,Suzuki SX4 I (Classic) Рестайлинг,2013,130500,700000,1.6,112,0,автомат,хэтчбек 5 дв.,...,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0


Транслитерация названий колонок, записанных кириллицей:

In [237]:
new_cols = []

In [238]:
for c in df.columns.to_list():
    try:
        new_cols.append(transliterate.translit(c, reversed=True).replace(' ', '_').replace("'", ''))
    except BaseException:
        new_cols.append(c)

In [245]:
df = df.rename(columns=dict(zip(cols, new_cols)))

Создадим новые признаки с dummy-переменными для разных типов коробки передач.

In [259]:
temp_df = pd.get_dummies(df.drive_char_4).groupby(level=0).sum()

In [260]:
temp_df_cols = []

In [261]:
for c in temp_df.columns.to_list():
    try:
        temp_df_cols.append(transliterate.translit(c, reversed=True).replace(' ', '_').replace("'", ''))
    except BaseException:
        temp_df_cols.append(c)

In [262]:
temp_df = temp_df.rename(columns=dict(zip(temp_df.columns.to_list(), temp_df_cols)))

In [263]:
temp_df

,avtomat,variator,mehanika,robot
0,0,0,1,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,0,1,0
...,...,...,...,...
222,0,0,1,0
223,1,0,0,0
224,1,0,0,0
225,1,0,0,0


Соединяем таблицы, удаляем старый признак "drive_char_4", ставим на удобное место новые колонки.

In [264]:
df = pd.concat([df, temp_df], axis=1)
df = df.drop('drive_char_4', axis=1)

In [266]:
cols = df.columns.tolist()

In [277]:
cols = cols[:13] + cols[-4:] + cols[13:77]

In [278]:
df = df[cols]

In [283]:
df = df.replace({'хэтчбек 5 дв.': 0, 'компактвэн': 1, 'седан': 1})

Удаляем другие, ненужные признаки.

In [17]:
df = df.drop('names', axis=1)
df = df.drop('drive_char_7', axis=1)
df = df.drop('places', axis=1)

Сохраняем в конечный файл.

In [20]:
df.to_csv('cars_all1.csv', sep=';', encoding="utf-8-sig", index=False)